# JUICE RPWI HF SID2 (RAW): L1a High-res QL -- 2023/12/11

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_sid2_lib as juice_sid2

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
cal_mode = 0                            # 0: raw      1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 7.0                         # background: 7.5   CAL: 10
p_raw_min = 0.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max0, p_min0 = juice_cdf.cal_factors(0, cal_mode, cal, p_raw_max, p_raw_min)
print("conversion factor:", cf, "    MAX-min:", p_max0, p_min0)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'                         # Plot dump folder

# *** Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID02-high_20231019-1749.cdf'
data_name = 'SID02-high_20231020-1218.cdf'
data_name = 'SID02-high_20231020-1222.cdf'

# 202312 -- TEST
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202312_FS/cdf/'    # 202312_FS test folder
data_name_list = [#'SID2_20000101T004732-20000101T004929_0.cdf',
                  #'SID2_20000101T004934-20000101T005147_1.cdf',
                  #'SID2_20000101T005151-20000101T005357_2.cdf',
                  #'SID2_20000101T005401-20000101T005538_3.cdf',
                  'SID2_20000101T005543-20000101T005624_4.cdf',
                 ]

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    cdf = pycdf.CDF(cdf_file)
    data1 = juice_sid2.hf_sid2_read(cdf, cf)

    if i==0:
        # print(data1.Eu_i)
        data = data1
    else:
        # print(data1.Eu_i)
        data = juice_sid2.hf_sid2_add(data, data1)
        print(data.Eu_i.shape)

In [ ]:
"""
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid2.juice_getdata_hf_sid2(cdf, cf)
"""

date1 = data.epoch[0]
date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]
date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

print(data.Eu_i.shape)
n_time0 = data.Eu_i.shape[0]
n_freq0 = data.Eu_i.shape[1]
# n_samp0 = data.Eu_i.shape[2]
f_min0 = data.frequency[0][0] # [0]
f_max0 = (max(np.ravel(data.frequency)))

In [ ]:
# Mode 
N_ch0 = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
print("Ch:", N_ch0, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("Num-samples:", n_time0, "   Num-Frequency", n_freq0) # , "   Length:", n_samp0)
print("Frequency, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0], data.freq_step[0][0])
#print("Frequency, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0][0], data.freq_step[0][0][0])
print("Time-length:", data.time[0][0], "sec in 1-sweep")
# print("Time-length:", data.time[0][0][n_samp0-1], "sec in 1-sweep")

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(5, 1, 1);  ax2 = fig.add_subplot(5, 1, 2);  ax3 = fig.add_subplot(5, 1, 3)
ax4 = fig.add_subplot(5, 1, 4);  ax5 = fig.add_subplot(5, 1, 5)

ax1.plot(np.ravel(data.Eu_i[:][:]), '-r', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(data.Eu_q[:][:]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[:][:]), '-r', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(data.Ev_q[:][:]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[:][:]), '-r', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(data.Ew_q[:][:]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency),  '-b', linewidth=.5, label='Frequency');
ax4.plot(np.ravel(data.freq_step*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width*10), ':b', linewidth=1.0, label='width*10')
ax4.plot(np.ravel(data.sweep_start)*data.frequency[0][-1], '-r', linewidth=.5, label='Sweep Start');
#ax4.plot(np.ravel(data.sweep_start)*data.frequency[0][-1][0], '-r', linewidth=.5, label='Sweep Start');
ax5.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  ax5.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
"""
# range: Y-axis
ylim=[-10**(p_max0-4.5), 10**(p_max0-4.5)]
ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
ylim=[f_min0, f_max0]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)